0 - create dataset  
1 - run experiment  
2 - plot counterfactuals  
3 - plot embeddings  
4 - plot results  

In [1]:
import pickle

from src.evaluation.evaluator_manager import EvaluatorManager
from src.evaluation.evaluator_manager_do import EvaluatorManager as PairedEvaluatorManager
from src.evaluation.evaluator_manager_triplets import EvaluatorManager as TripletsEvaluatorManager

from src.utils.context import Context

from utils_martina.my_utils import *

In [2]:
# Set patient and record (same channels)

# observations = ["chb01_03", "chb01_04", "chb01_15", "chb02_16", "chb02_19", "chb03_01", "chb03_02", "chb03_03", "chb04_28", "chb04_08", "chb06_01", "chb06_04"]

observations = ["chb01_03", "chb01_04", "chb01_15", "chb01_16", "chb01_18", "chb01_21", "chb01_26"]
# observations = ["chb02_16", "chb02_19"]
# observations = ["chb03_01", "chb03_02", "chb03_03", "chb03_04", "chb03_34", "chb03_35", "chb03_36"]
# observations = ["chb04_28", "chb04_08"]
# observations = ["chb05_06", "chb05_13", "chb05_16", "chb05_17", "chb05_22"]
# observations = ["chb06_01", "chb06_04"] #, "chb06_09", "chb06_10", "chb06_13", "chb06_18", "chb06_24"]

# observations = ["PN00_1", "PN00_2", "PN00_4", "PN00_5"]

create_dataset_json(observations)

In [3]:
logs_path = "..\\..\\explainability\GRETEL-repo\\output\\logs\\"
eval_manager_path = "..\\..\\explainability\GRETEL-repo\\output\\eval_manager\\"

## Removing cache dataset

In [4]:
remove_cache_dataset = True

if remove_cache_dataset:
    folder = ".\data\cache\datasets"

    for file in os.listdir(folder):
        if file.startswith("GCS-"):  # Checks if the file begins with "GCS-"
            cache_path = os.path.join(folder, file)
            if os.path.isfile(cache_path):
                os.remove(cache_path)
                print(f"Removed: {cache_path}")

Removed: .\data\cache\datasets\GCS-21f4bf75c5ac69d98dc7653d96c35ad9


## Run Gretel experiment

In [5]:
# NOTICE: If it crashes, check if there are any OracleTorch files from the future!

In [6]:
path = 'config/GCS-GCN.jsonc'
# path = 'config/GCS-SVM.jsonc'
# path = 'config/GCS-KNN.jsonc'

print(observations)

print(f"Generating context for: {path}")
context = Context.get_context(path)
context.run_number = -1

context.logger.info(f"Executing: {context.config_file} Run: {context.run_number}")
context.logger.info(
    "Creating the evaluation manager......................................................."
)

if 'doe-triplets' in context.conf:
    context.logger.info("Creating the TRIPLET evaluators........................................................")
    eval_manager = TripletsEvaluatorManager(context)
if 'do-pairs' in context.conf:
    context.logger.info("Creating the PAIRED evaluators...............................................................") ### Usiamo questo
    eval_manager = PairedEvaluatorManager(context)
else:
    context.logger.info("Creating the evaluators...............................................................")
    eval_manager = EvaluatorManager(context)

context.logger.info(
    "Evaluating the explainers............................................................."
)

eval_manager.evaluate()

# Additional stuff
# context.logger.info(f"graph_ids: {eval_manager._evaluators[0].get_instance_and_counterfactual_ids()}") # NON È STATO SISTEMATO PER CONTEMPLARE IL CASO DI PIU' PAZIENTI
context.logger.info(f"graph_metrics: {eval_manager._evaluators[0].get_instance_and_counterfactual_graph_metrics()}")

['chb01_03', 'chb01_04', 'chb01_15', 'chb01_16', 'chb01_18', 'chb01_21', 'chb01_26']
Generating context for: config/GCS-GCN.jsonc
2025-03-25 14:22:34,981 | INFO | 16560 - Executing: config/GCS-GCN.jsonc Run: -1
2025-03-25 14:22:35,012 | INFO | 16560 - Creating the evaluation manager.......................................................
2025-03-25 14:22:35,044 | INFO | 16560 - Creating the PAIRED evaluators...............................................................
2025-03-25 14:22:35,132 | INFO | 16560 - Creating: GCS-7461d1da0d4c1403016b336840abe8b6
2025-03-25 14:22:35,132 | INFO | 16560 - Instantiating: src.dataset.generators.gcs.GCS
2025-03-25 14:22:44,395 | INFO | 16560 - Saved: GCS-7461d1da0d4c1403016b336840abe8b6
2025-03-25 14:22:44,442 | INFO | 16560 - Created: GCS-7461d1da0d4c1403016b336840abe8b6
2025-03-25 14:22:44,510 | INFO | 16560 - Instantiating: src.oracle.nn.gcn.DownstreamGCN
2025-03-25 14:22:44,510 | INFO | 16560 - Instantiating: torch.optim.RMSprop
2025-03-25 14:2

c:\Users\zanno\explainability\GRETEL-repo\src\explainer\generative\gans\graph\res_gen.py:64: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3687.)
  edge_weights = torch.add(edge_weights, rebuild_adj_matrix(len(node_features), edge_list, edge_attr.T, self.device))
c:\Users\zanno\explainability\GRETEL-repo\src\explainer\generative\gans\graph\model.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  D_loss = torch.mean(self.loss_fn(y_pred.squeeze().double(), y_batch.double())

2025-03-25 15:00:11,418 | INFO | 16560 - Epoch 2	 Loss_D =  0.6987	 Loss_G =  0.6971
2025-03-25 15:00:11,524 | INFO | 16560 - Epoch 3	 Loss_D =  0.6845	 Loss_G =  0.6966
2025-03-25 15:00:11,619 | INFO | 16560 - Epoch 4	 Loss_D =  0.6884	 Loss_G =  0.6968
2025-03-25 15:00:11,715 | INFO | 16560 - Epoch 5	 Loss_D =  0.6912	 Loss_G =  0.6973
2025-03-25 15:00:11,809 | INFO | 16560 - Epoch 6	 Loss_D =  0.6897	 Loss_G =  0.6969
2025-03-25 15:00:11,914 | INFO | 16560 - Epoch 7	 Loss_D =  0.6971	 Loss_G =  0.6978
2025-03-25 15:00:12,011 | INFO | 16560 - Epoch 8	 Loss_D =  0.6918	 Loss_G =  0.6966
2025-03-25 15:00:12,120 | INFO | 16560 - Epoch 9	 Loss_D =  0.6915	 Loss_G =  0.6962
2025-03-25 15:00:12,198 | INFO | 16560 - Epoch 10	 Loss_D =  0.6874	 Loss_G =  0.6963
2025-03-25 15:00:12,308 | INFO | 16560 - Epoch 11	 Loss_D =  0.6912	 Loss_G =  0.6986
2025-03-25 15:00:12,399 | INFO | 16560 - Epoch 12	 Loss_D =  0.7008	 Loss_G =  0.6970
2025-03-25 15:00:12,494 | INFO | 16560 - Epoch 13	 Loss_D =  0

In [7]:
# Save eval_manager to file
file_name = get_most_recent_file(logs_path).split('.')[0]

with open(eval_manager_path + file_name + ".pkl", 'wb') as f:
    pickle.dump(eval_manager, f)

## _______________________________________________________________

In [8]:
import datetime

now = datetime.datetime.now()
print("Last full run:", now.strftime("%d/%m/%Y, ore %H:%M"))

Last full run: 25/03/2025, ore 15:07


In [9]:
import winsound

winsound.Beep(440, 500)
winsound.Beep(400, 500)
winsound.Beep(440, 500)

In [10]:
# from src.data_analysis.future.data_analyzer import DataAnalyzer as data_analyzer

# results_path = "output\\results"
# stats_file_path = "output"
# res = data_analyzer.create_aggregated_dataframe(results_path)
# res.to_csv(stats_file_path)
# res